In [ ]:
import pandas as pd
#data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new 1110 for colab.csv')
!pip install xgboost==1.6.1
import pickle
from matplotlib import pyplot

In [ ]:

data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new 1110 for colab area amplified arranged for colab.csv')


In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:5].values
Y = data.iloc[:,5:].values

train_data,test_data,train_targets,test_targets=train_test_split(X,Y,test_size=0.11,random_state=42)
print(len(train_data))
print(len(test_data))
print(test_data[0])

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

def mae(actual,pred):
     return (mean_absolute_error(actual,pred))
from sklearn.metrics import mean_absolute_percentage_error
def mape(actual,pred):
    calc=np.mean(np.absolute((actual-pred)*100/actual))
    return calc

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor

gbrm=MultiOutputRegressor(GradientBoostingRegressor())

taking default params

In [ ]:
defparam=gbrm.get_params()
print(defparam)

In [ ]:
gbrm.fit(train_data,train_targets)
predgb=gbrm.predict(test_data)

print(predgb)
print(test_targets)
print("mae:",mae(test_targets,predgb))
print("mape:",mape(test_targets,predgb))


In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(test_targets,predgb,multioutput="variance_weighted")
print(r2)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_targets,predgb))
print("rmse:",rmse)

individual parameter settings: mixed, but max_depth is altered

In [ ]:
gbrmnew=MultiOutputRegressor(GradientBoostingRegressor(max_features=1,max_depth=4,min_samples_leaf=1,min_samples_split=2,n_estimators=1000,learning_rate=0.3))  
gbrmnew.fit(train_data,train_targets)
predgbnew=gbrmnew.predict(test_data)

print(predgbnew)
print(test_targets)
print("mae:",mae(test_targets,predgbnew))
print("mape:",mape(test_targets,predgbnew))

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(test_targets,predgbnew,multioutput="variance_weighted")
print(r2)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_targets,predgbnew))
print("rmse:",rmse)

In [ ]:
# create an iterator object with write permission - model.pkl
with open("/content/drive/MyDrive/Colab Notebooks/gbr_avggc1110areanorm.pkl", 'wb') as files:
    pickle.dump(gbrmnew, files)

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(test_targets,predgbnew,multioutput="variance_weighted")
print(r2)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_targets,predgbnew))
print("rmse:",rmse)

In [ ]:
#gbr
import pickle
with open("/content/drive/MyDrive/Colab Notebooks/gbr_avggc1110areanorm.pkl" , 'rb') as f:
    new_model = pickle.load(f)

predgbnew2=new_model.predict(test_data)

print(predgbnew2)
print(test_targets)
print("mae:",mae(test_targets,predgbnew2))
print("mape:",mape(test_targets,predgbnew2))
from sklearn.metrics import r2_score
r2=r2_score(test_targets,predgbnew2,multioutput="variance_weighted")
print(r2)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_targets,predgbnew2))
print("rmse:",rmse)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=10, random_state=1, shuffle=True)
cvsxgbr=cross_val_score(gbrmnew,train_data,train_targets,cv=cv,n_jobs=-1,scoring="neg_mean_absolute_error")

print(cvsxgbr)

In [ ]:
from numpy import mean,absolute
scores=mean(absolute(cvsxgbr))

print(scores)

Randomsearch cv


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 20000, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5,8, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2,3, 4]

# Create the random grid
random_grid = {'estimator__n_estimators': n_estimators,
               'estimator__max_features': max_features,
               'estimator__max_depth': max_depth,
               'estimator__min_samples_split': min_samples_split,
               'estimator__min_samples_leaf': min_samples_leaf,
              }
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
gbr_random = RandomizedSearchCV(estimator = gbrm, param_distributions = random_grid, n_iter =1000, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
gbr_random.fit(train_data, train_targets)

In [ ]:
print(gbr_random.best_params_)

{'estimator__n_estimators': 7494, 'estimator__min_samples_split': 8, 'estimator__min_samples_leaf': 1, 'estimator__max_features': 'sqrt', 'estimator__max_depth': 10} 10/28

mae: 2.360238787763666
mape: 16.33349565631594

In [ ]:
gbrnew=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=7494,min_samples_split=8,min_samples_leaf=1,max_features='sqrt',max_depth=10))
gbrnew.fit(train_data,train_targets)
predgbnew=gbrnew.predict(test_data)

print(predgbnew)
print(test_targets)
print("mae:",mae(test_targets,predgbnew))
print("mape:",mape(test_targets,predgbnew))